<div style="color: rgba(14, 90, 132, 1); vertical-align:middle; text-align: center; font-family: Montserrat;">
    <div style="font-size: 40px; font-weight: bold;">
        Análise Eleições Guarulhos - 1º Turno
    </div>
    <img src="https://solaris-mh8n.onrender.com/static/img/logo_solaris3.png" align="middle" width="30%" height="30%"/>
    <div style="font-size: 20px; font-style: italic; font-family: Montserrat;">
        SOLARIS SISTEMAS <br /> - SS -
    </div>
</div>

In [1]:
import pandas as pd
import warnings
import os
import plotly.graph_objects as go
import folium
import json
import duckdb

from folium.plugins import Draw, BeautifyIcon, MiniMap, MarkerCluster, TimestampedGeoJson, Geocoder
from sqlalchemy import create_engine, text

warnings.filterwarnings("ignore")

In [2]:
dir_tse = 'D:/dabase_loads/tse_raw'

In [3]:
zona = pd.read_csv(os.path.join(dir_tse, 'eleitorado_local_votacao_2024.csv'), sep=';', encoding='latin1') 

In [4]:
zona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600374 entries, 0 to 600373
Data columns (total 41 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   DT_GERACAO                    600374 non-null  object 
 1   HH_GERACAO                    600374 non-null  object 
 2   AA_ELEICAO                    600374 non-null  int64  
 3   DT_ELEICAO                    600374 non-null  object 
 4   DS_ELEICAO                    600374 non-null  object 
 5   NR_TURNO                      600374 non-null  int64  
 6   SG_UF                         600374 non-null  object 
 7   CD_MUNICIPIO                  600374 non-null  int64  
 8   NM_MUNICIPIO                  600374 non-null  object 
 9   NR_ZONA                       600374 non-null  int64  
 10  NR_SECAO                      600374 non-null  int64  
 11  CD_TIPO_SECAO_AGREGADA        600374 non-null  int64  
 12  DS_TIPO_SECAO_AGREGADA        600374 non-nul

In [5]:
gru = zona.query("SG_UF == 'SP' and NM_MUNICIPIO == 'GUARULHOS'")
gru['NR_LATITUDE'] = gru['NR_LATITUDE'].astype(float)
gru['NR_LONGITUDE'] = gru['NR_LONGITUDE'].astype(float)
gru.head()

,DT_GERACAO,HH_GERACAO,AA_ELEICAO,DT_ELEICAO,DS_ELEICAO,NR_TURNO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,...,DS_SITU_LOCALIDADE,CD_SITU_SECAO_ACESSIBILIDADE,DS_SITU_SECAO_ACESSIBILIDADE,QT_ELEITOR_SECAO,QT_ELEITOR_ELEICAO_FEDERAL,QT_ELEITOR_ELEICAO_ESTADUAL,QT_ELEITOR_ELEICAO_MUNICIPAL,NR_LOCAL_VOTACAO_ORIGINAL,NM_LOCAL_VOTACAO_ORIGINAL,DS_ENDERECO_LOCVT_ORIGINAL
13985,10/10/2024,02:00:50,2024,06/10/2024,1º Turno,1,SP,64777,GUARULHOS,394,...,Ativo,0,Sem acessibilidade,387,0,0,385,1414,EE. PARQUE SANTOS DUMONT,"RUA OLHO D'ÁGUA DOS BORGES, 75"
13986,10/10/2024,02:00:50,2024,06/10/2024,1º Turno,1,SP,64777,GUARULHOS,278,...,Ativo,1,Com acessibilidade,345,0,0,345,2054,COLEGIO GUILHERME DE ALMEIDA,"AVENIDA EMÍLIO RIBAS, 855"
13989,10/10/2024,02:00:50,2024,06/10/2024,1º Turno,1,SP,64777,GUARULHOS,176,...,Ativo,0,Sem acessibilidade,349,0,0,349,1961,E.E. LYDIA KITZ MOREIRA,"RUA DOREZOPOLIS, 294"
13991,10/10/2024,02:00:50,2024,06/10/2024,1º Turno,1,SP,64777,GUARULHOS,185,...,Ativo,0,Sem acessibilidade,385,0,0,385,1171,EE. PROFESSOR MÁRIO NAKATA,"RUA EURACHIO MAURICIO, SN"
13992,10/10/2024,02:00:50,2024,06/10/2024,1º Turno,1,SP,64777,GUARULHOS,278,...,Ativo,0,Sem acessibilidade,270,0,0,271,1155,EE. PROFESSOR ENNIO CHIESA,"RUA MARIANO MANZONE, 88"


In [6]:
title_html = '<h3 align="center" style="font-size:30px;font-weight: bold;">ZONAS ELEITORAIS em Guarulhos</h3>'
mapa = folium.Map(location=[-23.397106352886972, -46.43988504955854], tiles="OpenStreetMap", zoom_start=11, control_scale = True, scrollWheelZoom=True, dragging=True)
mapa.get_root().html.add_child(folium.Element(title_html))

In [7]:
# Camadas

## Aeroporto
f_aeroporto = 'C:\\py\\sdceti\\Geo\\GeoJson\\aeroporto_limite.geojson'
aeroporto = json.load(open(f_aeroporto))
style1 = {'fillColor': 'yellow', 'color': 'blue', 'weight': 1.0}
folium.GeoJson(aeroporto, style_function=lambda x:style1).add_to(mapa)

## Bairros
f_bairros = 'C:\\py\\sdceti\\Geo\\GeoJson\\bairros_sem_aeroporto.geojson'
bairros = json.load(open(f_bairros, encoding='utf8'))
style2 =  {'color': 'black', 'weight': 2.0, 'fill_opacity': 0.3}
folium.GeoJson(bairros, tooltip=folium.GeoJsonTooltip(fields=['BAIRRO1']), popup=folium.GeoJsonPopup(fields=['BAIRRO1']), style_function=lambda x:style2).add_to(mapa)

In [ ]:
gru['NR_LATITUDE'].count()

In [8]:
gru.groupby('NR_ZONA')[['NR_LATITUDE', 'NR_LONGITUDE']]

In [ ]:
%%time

cor = 'blue'
tam_bot = 3

for row in gru.groupby('NR_ZONA')[['NR_LATITUDE', 'NR_LONGITUDE']].mean().reset_index().itertuples():
    icon_circle = BeautifyIcon(icon_shape='circle-dot', border_color=cor, border_width=tam_bot)
    folium.Marker(location = [row.NR_LATITUDE, row.NR_LONGITUDE], popup = '{}'.format(row.NR_ZONA), tooltip = 'ZONA: {}'.format(row.NR_ZONA), icon = icon_circle, fill = True, fillcolor = cor, color = cor, fill_opacity=1).add_to(mapa)
    


In [ ]:
mapa

In [ ]:
%%time

cor = 'blue'
tam_bot = 3

for row in gru.itertuples():
    icon_circle = BeautifyIcon(icon_shape='circle-dot', border_color=cor, border_width=tam_bot)
    folium.Marker(location = [row.NR_LATITUDE, row.NR_LONGITUDE], popup = '{}'.format(row.NR_ZONA), tooltip = 'ZONA: {} SEÇÃO: {} BAIRRO: {}'.format(row.NR_ZONA, row.NR_SECAO, row.NM_BAIRRO), icon = icon_circle, fill = True, fillcolor = cor, color = cor, fill_opacity=1).add_to(mapa)
    

In [ ]:
mapa

In [ ]:
voto = pd.read_csv(os.path.join(dir_tse, 'votacao_partido_munzona_2024_SP.csv'), sep=';', encoding='latin1') 

In [ ]:
voto.info()

In [ ]:
voto_gru = voto.query("SG_UF == 'SP' and NM_MUNICIPIO == 'GUARULHOS'")

In [ ]:
voto_gru.info()

In [ ]:
voto_gru.head()

In [ ]:
voto_gru.groupby('DS_CARGO').DS_CARGO.count()

In [ ]:
voto_gru.query("DS_CARGO == 'Prefeito'").iloc[0]

In [ ]:
voto_gru.query("DS_CARGO == 'Prefeito'").groupby('NR_PARTIDO')['QT_VOTOS_NOMINAIS_VALIDOS'].sum()

In [ ]:
%%time

dir_tse = 'D:/dabase_loads/tse_raw'
local = pd.read_csv(os.path.join(dir_tse, 'eleitorado_local_votacao_2024.csv'), sep=';', encoding='latin1') 
local.columns = local.columns.str.lower()
voto = pd.read_csv(os.path.join(dir_tse, 'votacao_partido_munzona_2024_SP.csv'), sep=';', encoding='latin1') 
voto.columns = voto.columns.str.lower()

con = duckdb.connect('d:/db/sdceti03.duckdb')
con.sql('create table tse.local as select * from local')
con.sql('create table tse.voto as select * from voto')

In [ ]:
del con